In [3]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
import json

_ = load_dotenv(find_dotenv())

client = OpenAI()

### Steps

#### 1. Define `Functions`
#### 2. Create an `Assitant` and register Functions
#### 3. Create a `Thread`
#### 4. Add a `Message` to the Thread
#### 5. `Run` the Assistant
#### 6. Polling for updates and `Calling` Functions

## 1. Define Functions

In [4]:
# Define the functions


def getCurrentWeather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location):
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location

## 2. Create an Assistant and register functions to it

In [5]:
assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo-1106",
    #  register functions in tools
    tools=[{
        "type": "function",
        "function": {
            # first function
            "name": "getCurrentWeather",
            "description": "Get the weather in location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
                    "unit": {"type": "string", "enum": ["c", "f"]}
                },
                "required": ["location"]
            }
        }
    }, {
        "type": "function",
        "function": {
            # second function
            "name": "getNickname",
            "description": "Get the nickname of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
                },
                "required": ["location"]
            }
        }
    }]
)

dict(assistant)


{'id': 'asst_nS4L4Nq51yoJxP2h3dbZwkPw',
 'created_at': 1704128041,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': None,
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['location']}, description='Get the nickname of a city'), type='function')]}

## 3. Create a Thread

In [6]:
thread = client.beta.threads.create()

print(thread)

Thread(id='thread_ZnRRo7Ly5Xg47kIbfNMzGnnv', created_at=1704128042, metadata={}, object='thread')


## 4. Add a Message

In [7]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What is the temperature in Karachi"
)

dict(message)

{'id': 'msg_4MJdKXJp7tMSvoNxw06FtcuS',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='What is the temperature in Karachi'), type='text')],
 'created_at': 1704128042,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_ZnRRo7Ly5Xg47kIbfNMzGnnv'}

## 5. Run the Assistant

In [8]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,    
)

dict(run)

{'id': 'run_u2DXsx2aPg0uceEcGYWuWFKH',
 'assistant_id': 'asst_nS4L4Nq51yoJxP2h3dbZwkPw',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704128043,
 'expires_at': 1704128643,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_ZnRRo7Ly5Xg47kIbfNMzGnnv',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object

## 6. Polling for Updates

In [15]:
import time

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))

available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname
} 

# Loop until the run completes or requires action
while True:

    runStatus = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id
    )

    # show_json("Run Steps:", run_steps)
    print("Run Status: ", runStatus.status)

    # This means run is making a function call
    if runStatus.status == "requires_action":
        print("Status when requires_action: ",runStatus.status)    
        show_json("submit_tool_outputs", runStatus.required_action)

        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls

            tool_outputs = []
            
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)

                if function_name in available_functions:

                    function_to_call = available_functions[function_name]
                    print(function_to_call, function_to_call.__name__ == "getCurrentWeather",
                          "================================================================")

                    if function_to_call.__name__ == "getCurrentWeather":

                        response = function_to_call(
                            location = function_args.get("location"),
                            unit = function_args.get("unit")
                        )

                        tool_outputs.append({
                            "tool_call_id": toolcall.id,
                            "output": response
                        })

                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                            location = function_args.get("location")
                        )
                        tool_outputs.append({
                            "tool_call_id": toolcall.id,
                            "output": response,
                        })

            print(tool_outputs, ">>>>>")

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        
        messages = client.beta.threads.messages.list(
            thread_id = thread.id
        )
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
        print("Run failed.")
        break

    elif run.status in ["in_progress", "queued"]:
        print(f"Run is {run.status}. Waiting...")
        time.sleep(5)  # Wait for 5 seconds before checking again

    else:
        print(f"Unexpected status: {run.status}")
        break


Run Status:  failed
Run is queued. Waiting...
Run Status:  failed
Run is queued. Waiting...
Run Status:  failed
Run is queued. Waiting...
Run Status:  failed
Run is queued. Waiting...
Run Status:  failed
Run is queued. Waiting...


KeyboardInterrupt: 

## More

In [ ]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

dict(run)

{'id': 'run_8pzdMIlON5jAXw6ubq3R9Zm4',
 'assistant_id': 'asst_1A5urdATFYvPlWV0a3pH9BbW',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704047583,
 'expires_at': 1704048183,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1704047583,
 'status': 'in_progress',
 'thread_id': 'thread_kiNCVfXpNLwNgZeHgBk3AitW',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'typ

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
)

print(messages)

for message in messages.data:
    print(message)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_FNZNrcPsJesWEhCgkChDnEhX', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the temperature in Karachi'), type='text')], created_at=1704047582, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_kiNCVfXpNLwNgZeHgBk3AitW')], object='list', first_id='msg_FNZNrcPsJesWEhCgkChDnEhX', last_id='msg_FNZNrcPsJesWEhCgkChDnEhX', has_more=False)
ThreadMessage(id='msg_FNZNrcPsJesWEhCgkChDnEhX', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the temperature in Karachi'), type='text')], created_at=1704047582, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_kiNCVfXpNLwNgZeHgBk3AitW')
